Non complet : 

Vérifier d'où viennent les erreurs, à voir plus tard. 

In [1]:
import polars as pl 
import gzip 
import pandas as pd 
import xmltodict
import glob
import os 
import time 
import polars as pl 
import duckdb
import shutil 
import pyarrow.parquet as pq
import pyarrow as pa
import traceback

In [3]:
CHEMIN_ENTREE = './path/to/dossier_contenant_les_xml/'
CHEMIN_SORTIE = './path/to/dossier_qui_contiendra_les_parquets/'

In [3]:

def _recherche_id_dans_parquet(chemin_parquet_db):
 
 conduck = duckdb.connect(database=':memory:', read_only=False)
 source_duck = conduck.read_parquet(f'{chemin_parquet_db}')
 requete = conduck.execute(''' 
    SELECT DISTINCT Id_Fichier 
    FROM source_duck''')
 ligne_sql_int = [int(x[0]) for x in requete.fetchall()]
 conduck.close()
 return ligne_sql_int

def parse_fichier(chemin) : 
 '''Ouvre et parse le fichier gzip'''
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

def recherche_id_fichier(chemin_parquet) :
 ''' Pas encore utilisée, permet de récupérer les ID dans le parquet contenant les 
 données déjà traitées '''
 conduck = duckdb.connect(database=':memory:', read_only=False)
 docubudg_t = conduck.read_parquet(chemin_parquet)
 requete_duckdb = ''' 
 SELECT
    DISTINCT Id_Fichier
 FROM 
    docubudg_t
 '''
 result_requete= conduck.execute(requete_duckdb).fetchdf()
 liste_id = result_requete['Id_Fichier'].to_list()
 conduck.close()
 return liste_id

def _isolement_id(fichier) : 
 '''Extrait l'id du nom du fichier pour la liste comprehension de securité

 ATTENTION, le premier split / va changer si on l'applique sur du minio '''
 val_id_fichier_source = fichier.split("/")[-1].split('.')[0]
 if '-' in val_id_fichier_source : 
  val_id_fichier = val_id_fichier_source.split('-')[1]
 else : 
  val_id_fichier= val_id_fichier_source
 return val_id_fichier


def nettoyage_V(dataframe : pd.DataFrame) -> pd.DataFrame :
 ''' Permet de supprimer les @V des colonnes à l'exception de MtSup et CaracSup'''

 nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
 for col in dataframe.columns : 
  if col in ['MtSup', 'CaracSup'] : 
   dataframe[col] = dataframe[col].astype(str) 
  else :
   dataframe[col] = dataframe[col].apply(nettoyage)
 dataframe_propre = dataframe.reset_index(drop=True)
 return dataframe_propre

def extraction_annexe(chemin_annexe, dict_metadonnees) : 
 liste_annexe = []
 for row in chemin_annexe : 
  liste_par_ligne = {}
  for a, b in row.items() : 
   liste_par_ligne.update({a : b.get('@V')})
   liste_par_ligne.update(dict_metadonnees)
  liste_annexe.append(liste_par_ligne)
 return liste_annexe 

def travail_mtsup_ligne(ligne_budg) :
 type_m = ligne_budg.get('MtSup')
 if type_m is None : 
  pass 
 else : 
  if isinstance(type_m, dict) : 
   ligne_budg[type_m.get('@Code', '')] = {'@V' : type_m.get('@V', '')}
  elif isinstance(type_m, list) :
    for z in type_m : #z c'est un dict contenant un V et un Code 
      ligne_budg[z.get('@Code', '')] = {'@V' : z.get('@V', '')}
 return ligne_budg

def travail_caracsup_ligne(ligne_budg) : 
 type_c = ligne_budg.get('CaracSup')
 if type_c is None : 
  pass 
 else : 
  if isinstance(type_c, dict) : 
   ligne_budg[type_c.get('@Code', '')] = {'@V' : type_c.get('@V', '')}
  elif isinstance(type_c, list) : 
   for ii in type_c : 
    ligne_budg[ii.get('@Code', '')] = {'@V' : ii.get('@V', '')}
 return ligne_budg


def xml_to_parquet(chemin_xml_entree, chemin_sortie) : 
 #
 #REMPLACER chemin_xml_entree_glob par liste_fichiers_safe
 #
 #
 chemin_xml_entree_glob = glob.glob(os.path.join(chemin_xml_entree, "*.gz"))
 #liste_verif_parquet = _recherche_id_dans_parquet(chemin_parquet_db= chemin_parquet_verif)
 #liste_fichiers_safe = [chemin_fichier for chemin_fichier in chemin_xml_entree_glob if int(_isolement_id(chemin_fichier)) not in liste_verif_parquet]
 for fichier in chemin_xml_entree_glob : 
  try : 
   id_fichier = _isolement_id(fichier)
   dico = parse_fichier(fichier)
   metadonnees = dico['DocumentBudgetaire']['EnTeteDocBudgetaire']
   metadonnees.update(dico['DocumentBudgetaire']['Budget']['BlocBudget'])
   metadonnees.update(dico['DocumentBudgetaire']['Budget']['EnTeteBudget'])
   metadonnees['Id_Fichier'] = {'@V' : id_fichier}
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   if isinstance(docbase, list):
    for i in docbase : 
      i.update(metadonnees)
      i = travail_mtsup_ligne(i)
      i = travail_caracsup_ligne(i)
   elif isinstance(docbase, dict) : 
    docbase.update(metadonnees)
    docbase = travail_mtsup_ligne(docbase)
    docbase = travail_caracsup_ligne(docbase)
   df_base = pd.DataFrame(docbase)
   df_propre = nettoyage_V(df_base)
   df_propre.to_parquet(f'{chemin_sortie}{id_fichier}_parquet', engine='pyarrow')
   #shutil.move(fichier, './fichiers/done_xml/')
  except Exception as e : 
    print(f'Erreur fichier {fichier}, extraction impossible')
    #shutil.move(fichier, './fichiers/todo_xml/error/')
    print(f'Erreur complète : {repr(e)}')
    traceback.print_exc()
    continue 



In [4]:
def travail_mtsup_ligne(ligne_budg) :
 type_m = ligne_budg.get('MtSup')
 if type_m is None : 
  pass 
 else : 
  if isinstance(type_m, dict) : 
   ligne_budg[type_m.get('@Code', '')] = {'@V' : type_m.get('@V', '')}
  elif isinstance(type_m, list) :
    for z in type_m : #z c'est un dict contenant un V et un Code 
      ligne_budg[z.get('@Code', '')] = {'@V' : z.get('@V', '')}
 return ligne_budg

def travail_caracsup_ligne(ligne_budg) : 
 type_c = ligne_budg.get('CaracSup')
 if type_c is None : 
  pass 
 else : 
  if isinstance(type_c, dict) : 
   ligne_budg[type_c.get('@Code', '')] = {'@V' : type_c.get('@V', '')}
  elif isinstance(type_c, list) : 
   for ii in type_c : 
    ligne_budg[ii.get('@Code', '')] = {'@V' : ii.get('@V', '')}
 return ligne_budg


def extration_ligne_budget_temp(chemi_bloc_budget, dict_metadonnees) : 
 liste_budget = []
 for row in chemi_bloc_budget : 
  liste_par_ligne = {}
  for a, b in row.items() : 
    if a in ['MtSup', 'CaracSup'] : 
      pass 
    else : 
     liste_par_ligne.update({a : b.get('@V')})
     liste_par_ligne.update(dict_metadonnees)
     liste_budget.append(liste_par_ligne)
 return liste_budget 


In [5]:
pars1 = parse_fichier('../../../fichiers20/todo_xml_20/t10/612013.xml.gz')
budg1 = pars1['DocumentBudgetaire']['Budget']['LigneBudget']

dict1 = {'A' : 'A'}

In [116]:
l_liste = []
for i in budg1 : 
 for a,b in i.items() : 
    if isinstance(b, list) : 
        l_liste.append(i)

In [15]:
for a,b in budg1[0].items() : 
 dict_vide = {}
 if a == 'MtSup' :
  type_m = budg1[0].get('MtSup')
  if isinstance(type_m, dict) : 
    print(type_m)
    dict_vide.update({type_m.get('@Code') : type_m.get('@V')})
    print(dict_vide)

{'@V': '11962.00', '@Code': 'BudgetHorsRAR'}
{'BudgetHorsRAR': '11962.00'}


Methode fonctionnelle, pré vérification. 

In [138]:
liste_ligne = []

for ligne in budg1 : 
 dict_temp = {}
 for a,b in ligne.items() :
  if a == 'MtSup' :   
   dict_temp.update({a : b})
   if isinstance(b, dict) :  #Si MtSup est un dictionnaire
     dict_temp.update({b.get('@Code') : b.get('@V')})
   elif isinstance(b, list):  #Si MtSup est une liste 
     for j in b : 
       dict_temp.update({j.get('@Code') : j.get('@V')})
   else :  #Si MtSup n'existe pas 
     pass  
  elif a == 'CaracSup' : 
   dict_temp.update({a : b})
   if isinstance(b, dict) : 
     pass 
     dict_temp.update({b.get('@Code') : b.get('@V')})
   elif isinstance(b, list): 
     for j in b : 
       dict_temp.update({j.get('@Code') : j.get('@V')}) 
   else : 
     pass
  else : 
   dict_temp.update({a : b.get('@V')})
 liste_ligne.append(dict_temp)



Passage en fonction 

In [167]:

def extraction_ligne_budget_si_liste(dict_pars : list, dict_metadonnes : dict) -> list : 
 ''' Retire les données propre des dictionnaires ligne budget
 pour les envoyer dans une liste de dictionnaires ''' 
 liste_nouvelles_lignes = []
 for ligne in dict_pars : 
  dict_temp = {}
  for a,b in ligne.items() :
   if a == 'MtSup' :
    dict_temp.update({a : b})  
    if isinstance(b, dict) :  #Si MtSup est un dictionnaire
      dict_temp.update({b.get('@Code') : b.get('@V')})
    elif isinstance(b, list):  #Si MtSup est une liste 
      for j in b : 
        dict_temp.update({j.get('@Code') : j.get('@V')})
    else :  #Si MtSup n'existe pas 
      pass  
   elif a == 'CaracSup' : 
    dict_temp.update({a : b})  
    if isinstance(b, dict) : 
      pass 
      dict_temp.update({b.get('@Code') : b.get('@V')})
    elif isinstance(b, list): 
      for j in b : 
        dict_temp.update({j.get('@Code') : j.get('@V')}) 
    else : 
      pass
   else : #Tout ce qui n'est pas MtSup ou CaracSup
    dict_temp.update({a : b.get('@V')})
  liste_nouvelles_lignes.append(dict_temp)
 return liste_nouvelles_lignes 


def extraction_ligne_budget_si_dict(dict_pars : dict, dict_metadonnes : dict) -> list : 
 ''' Retire les données propre des dictionnaires ligne budget
 pour les envoyer dans une liste de dictionnaires ''' 
 liste_nouvelles_lignes = []
 dict_temp = {}
 for a,b in dict_pars.items() :
   if a == 'MtSup' :
    dict_temp.update({a : b})  
    if isinstance(b, dict) :  #Si MtSup est un dictionnaire
      dict_temp.update({b.get('@Code') : b.get('@V')})
    elif isinstance(b, list):  #Si MtSup est une liste 
      for j in b : 
        dict_temp.update({j.get('@Code') : j.get('@V')})
    else :  #Si MtSup n'existe pas 
      pass  
   elif a == 'CaracSup' : 
    dict_temp.update({a : b})  
    if isinstance(b, dict) : 
      pass 
      dict_temp.update({b.get('@Code') : b.get('@V')})
    elif isinstance(b, list): 
      for j in b : 
        dict_temp.update({j.get('@Code') : j.get('@V')}) 
    else : 
      pass
   else : #Tout ce qui n'est pas MtSup ou CaracSup
    dict_temp.update({a : b.get('@V')})
 liste_nouvelles_lignes.append(dict_temp)
 return liste_nouvelles_lignes 


In [140]:
liste_ligne[0]

{'Nature': '60632',
 'ContNat': '011',
 'ArtSpe': 'false',
 'CodRD': 'D',
 'MtBudgPrec': '11962.00',
 'MtPropNouv': '19335.00',
 'MtPrev': '19335.00',
 'CredOuv': '19335.00',
 'OpBudg': '0',
 'MtSup': {'@V': '11962.00', '@Code': 'BudgetHorsRAR'},
 'BudgetHorsRAR': '11962.00',
 'CaracSup': {'@V': 'F', '@Code': 'Section'},
 'Section': 'F'}

Donc, on a une nouvelle méthode d'extraction. Maintenant on va si ça a un impact 

In [168]:

def xml_to_parquet_v2(chemin_xml_entree, chemin_sortie) : 
 #
 #REMPLACER chemin_xml_entree_glob par liste_fichiers_safe
 #
 #
 chemin_xml_entree_glob = glob.glob(os.path.join(chemin_xml_entree, "*.gz"))
 #liste_verif_parquet = _recherche_id_dans_parquet(chemin_parquet_db= chemin_parquet_verif)
 #liste_fichiers_safe = [chemin_fichier for chemin_fichier in chemin_xml_entree_glob if int(_isolement_id(chemin_fichier)) not in liste_verif_parquet]
 for fichier in chemin_xml_entree_glob : 
  try : 
   id_fichier = _isolement_id(fichier)
   dico = parse_fichier(fichier)
   metadonnees = dico['DocumentBudgetaire']['EnTeteDocBudgetaire']
   metadonnees.update(dico['DocumentBudgetaire']['Budget']['BlocBudget'])
   metadonnees.update(dico['DocumentBudgetaire']['Budget']['EnTeteBudget'])
   metadonnees.update({'Id_Fichier' : id_fichier})
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']

   if isinstance(docbase, list) : 
    l_docbase = extraction_ligne_budget_si_liste(docbase, metadonnees)
    df_base = pd.DataFrame(l_docbase)
    df_propre = nettoyage_V(df_base)
    df_propre.to_parquet(f'{chemin_sortie}{id_fichier}_parquet', engine='pyarrow')
   else : 
    d_docbase = extraction_ligne_budget_si_dict(docbase, metadonnees)
    df_base = pd.DataFrame(l_docbase)
    df_propre = nettoyage_V(df_base)
    df_propre.to_parquet(f'{chemin_sortie}{id_fichier}_parquet', engine='pyarrow')
   #shutil.move(fichier, './fichiers/done_xml/')
  except Exception as e : 
    print(f'Erreur fichier {fichier}, extraction impossible')
    #shutil.move(fichier, './fichiers/todo_xml/error/')
    print(f'Erreur complète : {repr(e)}')
    traceback.print_exc()
    continue 


In [169]:
xml_to_parquet_v2('../../../data/todo_xml/', '../../../data/temp_test/')

Erreur fichier ../../../data/todo_xml/824064.xml.gz, extraction impossible
Erreur complète : AttributeError("'str' object has no attribute 'get'")


Traceback (most recent call last):
  File "/tmp/ipykernel_42292/2743745175.py", line 20, in xml_to_parquet_v2
    l_docbase = extraction_ligne_budget_si_liste(docbase, metadonnees)
  File "/tmp/ipykernel_42292/1470142132.py", line 28, in extraction_ligne_budget_si_liste
    dict_temp.update({a : b.get('@V')})
AttributeError: 'str' object has no attribute 'get'


Erreur fichier ../../../data/todo_xml/636984.xml.gz, extraction impossible
Erreur complète : AttributeError("'str' object has no attribute 'get'")


Traceback (most recent call last):
  File "/tmp/ipykernel_42292/2743745175.py", line 20, in xml_to_parquet_v2
    l_docbase = extraction_ligne_budget_si_liste(docbase, metadonnees)
  File "/tmp/ipykernel_42292/1470142132.py", line 28, in extraction_ligne_budget_si_liste
    dict_temp.update({a : b.get('@V')})
AttributeError: 'str' object has no attribute 'get'


Erreur fichier ../../../data/todo_xml/788349.xml.gz, extraction impossible
Erreur complète : AttributeError("'str' object has no attribute 'get'")


Traceback (most recent call last):
  File "/tmp/ipykernel_42292/2743745175.py", line 20, in xml_to_parquet_v2
    l_docbase = extraction_ligne_budget_si_liste(docbase, metadonnees)
  File "/tmp/ipykernel_42292/1470142132.py", line 28, in extraction_ligne_budget_si_liste
    dict_temp.update({a : b.get('@V')})
AttributeError: 'str' object has no attribute 'get'


KeyboardInterrupt: 

824064
636984
788349

In [151]:
parse_678365 = parse_fichier('../../../data/todo_xml/678365.xml.gz')
budg_678365 = parse_678365['DocumentBudgetaire']['Budget']['LigneBudget']
budg_678365

{'Nature': {'@V': '002'},
 'ArtSpe': {'@V': 'false'},
 'CodRD': {'@V': 'R'},
 'MtPrev': {'@V': '0.38'},
 'CredOuv': {'@V': '0.38'},
 'MtReal': {'@V': '0.38'},
 'OpBudg': {'@V': '0'},
 'CaracSup': {'@V': 'F', '@Code': 'Section'}}

In [166]:
fds = extraction_ligne_budget_si_dict(budg_678365, dict1 )
fds 

[{'Nature': '002',
  'ArtSpe': 'false',
  'CodRD': 'R',
  'MtPrev': '0.38',
  'CredOuv': '0.38',
  'MtReal': '0.38',
  'OpBudg': '0',
  'CaracSup': {'@V': 'F', '@Code': 'Section'},
  'Section': 'F'}]